In a previous notebook, the benefits and shortcomings of a Decision Tree model were evaluated by developing a `DecisionTree` and training it to predict whether a horse `lived`, `died`, or was `euthanized` given various symptoms and diagnostics. While the trained model was easy to interpret and fast at predicting the class of a novel input, the model ultimately struggled due in part to the sparsity of the dataset, the degree of noise during training, the presence of irrelevant variables, the small size of the training set, imbalanced classes, and high-cardinality features. The decision tree was slow to train and struggled to achieve better performance than a naive model which always classified inputs as belonging to the most frequent class.

For this dataset, a Random Forest model will be developed and evaluated by training it to accomplish a similar task - predict whether a person has heart disease given features such as their age, gender, and blood pressure. Before proceeding with the development of the Random Forest, however, basic attributes of the dataset will be assessed.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


df = pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")


print("Columns: {0}".format(df.columns.tolist()))
print("Row Count: {0}, Column Count: {1}".format(len(df), len(df.columns)))
print(df.head())
print("Number of NaN entries per column")
print(df.isnull().sum().div(len(df)))

The dataset contains 14 variables, one of which - `target` - is the output variable, while the other 13 variables are input variables. All input variables appear to be helpful in training a model, and no variables have any missing values, so it doesn't seem like any significant pre-processing such as imputation or dimensionality reduction will be necessary (although they may be beneficial in improving model performance later on). The relatively small size of the dataset may prove to be an obstacle in training an accurate model.

Are the output classes balanced? How many values does `target` have? 

In [ ]:
# While no significant pre-processing will be performed, the two categorical variables will be convereted from int to string. This will 
# make the decision tree simpler to develop.
df['cp'] = df['cp'].astype(str)
df['thal'] = df['thal'].astype(str)

print(df["target"].value_counts())

`target` is a binary variable, making the task a binary classification problem. Additionally, there is no significant class imbalance, with there being only 27 more instances of patients with cardiovascular disease than those without disease in the dataset. Class weights, undersampling, or other techniques to amend class immbalances therefore won't be necessary either.

Out of curiosity, let's construct a heapmap using `target` and `sex` to get an idea of their correlation.

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
import scipy.stats as stats

sn.heatmap(pd.crosstab(df["sex"], df["target"]), annot=True, cmap="YlGnBu")
plt.plot()
coef, p_value = stats.pearsonr(df["sex"], df["target"])
print("Matthew's Correlation Coefficient: {0}, p-value: {1}".format(coef, p_value))

The dataset has significantly more men than women: 203 to 96. Women in the dataset are also less likely to have heart disease than men, with 23% of women having heart disease while only 54% of men have heart disease. This agrees with some existing data that men are at a greater risk for heart disease than women - [According to the American Heart Association](https://www.ahajournals.org/doi/pdf/10.1161/CIR.0b013e3182009701), 35.0% of American women 20 and older have coronary heart disease, while 37.4% of American men 20 and older do. Also, Matthew's Correlation Coefficient, also known as the Phi Coefficient, has a statistically significant value of -0.281, suggesting the presence of a moderate negative correlation between the two variables. 

Now that some basic analysis of the dataset has been completed, and it doesn't seem any data pre-processing is blocking development of a model, a basic decision tree model will be developed and trained, so that it can later serve as the foundation of a Random Forest model.

In [ ]:
from collections import deque

class BDTNode:
    def __init__(self, df, target_col, level):
        """An individual node in a binary decision tree. It's either a leaf node or has two children, where true_child is the child of the node matching the BDTNode's condition, 
        and false_child is the child not matching the given condition.
         
        Keyword Arguments:
        df - The filtered DataFrame of this specific node, where the DataFrame will be a subset of the parent node's DataFrame after conditional filtering.
        target_col - The target variable to be used for training. As this tree is only intended for use with binary classification tasks, there can only be one target variable.
        level - The level of this node in the BinaryDecisionTree, with 0 denoting the root node.
        """
        self.df = df
        self.target = target_col
        self.level = level
        self.condition = None
        self.true_child = None
        self.false_child = None
        self.ig = 0
        
    def entropy(self):
        """Returns the entropy of this specific BDTNode.
        """
        return 0 if len(self.df[self.target].value_counts()) != 2 else (-self.df[self.target].value_counts()[0]/len(self.df) * np.log2(self.df[self.target].value_counts()[0]/len(self.df))) - (self.df[self.target].value_counts()[1]/len(self.df) * np.log2(self.df[self.target].value_counts()[1]/len(self.df)))
    
    def get_information_gain(self):
        """Returns the information gained by this specific node (not including grandchildren), or 0 if this node is a leaf node.
        """
        return 0 if not self.true_child else self.entropy() - ((self.true_child.entropy() * (len(self.true_child.df)/len(self.df))) + (self.false_child.entropy() * (len(self.false_child.df)/len(self.df))))
    
    def eval_conditional(self, lval, rval):
        if not self.condition:
            return False
        elif "==" in self.condition:
            return lval == rval
        else:
            return lval >= rval
        

class BinaryDecisionTree:
    def __init__(self, df, height, target_col):
        """A binary decision tree. The qualification of binary denotes that the decision tree is for binary classification tasks, and not categorical classification tasks, due to 
        how entropy is calculated in the BDTNode.
        
        Keyword Argumments:
        df - The DataFrame to use for training the BinaryDecisionTree. Note that the BinaryDecisionTree will not split the data up into testing and training sets - that must be done beforehand.
        height - The maximum height of the BinaryDecisionTree. This helps prevent overfitting in lieu of pruning.
        target_col - The target variable to be used for training. As this tree is only intended for use with binary classification tasks, there can only be one target variable.
        """
        self.height = height
        self.target = target_col
        self.root = BDTNode(df, self.target, 0)
        
    def train(self):
        """Trains the BinaryDecisionTree on the DataFrame specified by self.df. Builds branches using the self.branch(node) method.
        """
        q = deque([self.root])
        while q:
            curr_node = q.pop()
            if self._branch(curr_node):
                q.extendleft([curr_node.true_child, curr_node.false_child])
             
    def _branch(self, node):
        """A utility function which comprises the core logic of the BinaryDecisionTree. Branches a single BDTNode into two child nodes by greedily partitioning the DataFrame into
        two subsets based on the condtional split which results in the most information gain.
        
        Returns:
        True - if self._branch(node) paritioned node's data into two subsets
        False if self._branch(node) resulted in node becoming a leaf node, either due to the maximum height of the tree being reached or a failure to find a partition leading to
        information gain.
        """
        if node.level == self.height:
            return False
        df = node.df
        input_cols = df.columns.tolist()
        input_cols.remove(self.target)
        for col in input_cols:
            for unique_value in df[col].unique():
                subsets = [("{0} == {1}".format(col, unique_value), df.loc[df[col] == unique_value], df.loc[df[col] != unique_value]), 
                           ("{0} >= {1}".format(col, unique_value), df.loc[df[col] >= unique_value], df.loc[df[col] < unique_value])] if node.df[col].dtype in ["int64", 
                           "float64"] else [("{0} == {1}".format(col, unique_value), df.loc[df[col] == unique_value], df.loc[df[col] != unique_value])]
                for condition, c1, c2 in subsets:
                    old_true_child, old_false_child, old_condition = node.true_child, node.false_child, node.condition
                    node.condition = condition
                    node.true_child = BDTNode(c1, node.target, node.level + 1)
                    node.false_child = BDTNode(c2, node.target, node.level + 1)
                    ig = node.get_information_gain()
                    if ig > node.ig:
                        node.ig = ig
                    else:
                        node.condition = old_condition
                        node.true_child, node.false_child = old_true_child, old_false_child
        return node.ig != 0
    
    def predict(self, data):
        """Predicts the outcome for a specific data point.
        """
        curr_node = self.root
        while curr_node.true_child:
            col, eq, val = curr_node.condition.split()
            if curr_node.df[col].dtype in ["int64", "float64"]:
                curr_node = curr_node.true_child if curr_node.eval_conditional(data[col], float(val)) else curr_node.false_child
            else:
                curr_node = curr_node.true_child if curr_node.eval_conditional(data[col], val) else curr_node.false_child
        return ("The outcome for ({0}) is: {1}".format(str(data), curr_node.df[self.target].value_counts().idxmax()), curr_node.df[self.target].value_counts().idxmax())
    
    def predict_df(self, test_df, labels):
        """Makes a prediction for an entire testing set DataFrame and returns a confusion matrix of the predictions.
        
        Keyword Arguments:
        test_df - The testing set DataFrame
        labels - The ordering of labels for the confusion matrix to be returned
        """
        df = test_df.copy()
        df["prediction"] = df.apply(lambda row: self.predict(row.to_dict())[1], axis=1)
        return confusion_matrix(df[self.target], df["prediction"], labels=labels)
    
    
class Metrics:
    def __init__(self, cm):
        """A convenience class which helps display model performance metrics given a confusion matrix. 
        
        Keyword Arguments:
        cm - A 2D Array constituting a Confusion Matrix
        """
        self.cm = cm
        
    def get_accuracy(self):
        return sum([self.cm[i][i] for i in range(len(self.cm))]) / np.sum(self.cm)

    def get_recalls(self):
        return [self.cm[i][i]/np.sum(self.cm, axis=1)[i] for i in range(len(self.cm))]

    def get_precisions(self):
        return [self.cm[i][i]/np.sum(self.cm, axis=0)[i] for i in range(len(self.cm))]

    def get_weighted_f1(self):
        class_freq = np.sum(self.cm, axis=0)
        recalls, precisions = self.get_recalls(), self.get_precisions()
        return sum([(2 * frequency * precision * recall) / (precision + recall) for frequency, precision, recall in zip(class_freq, precisions, recalls)]) / sum(class_freq)
    
    def print(self):
        print("Confusion Matrix:\n{0}".format(self.cm))
        print("Accuracy: {0}".format(self.get_accuracy()))
        print("Recalls: {0}".format(self.get_recalls()))
        print("Precisions: {0}".format(self.get_precisions()))
        print("Weighted F1: {0}".format(self.get_weighted_f1()))
        

There isn't much additional to say about the above code outside of the provided comments. The `Metrics` class will be useful for judging the performance of the model after evaluation on the testing set. The two important methods in `BinaryDecisionTree` are `predict_df()` and `train()`, although each method uses one of the other two methods of the majority of their logic.

The specific metrics provided by `Metrics` apart from accuracy are to better compare different models and their respective performances, but more about the utility of these metrics and how decision trees are trained using entropy and information gain are described in a previous notebook of mine [here](https://www.kaggle.com/bullardla/kernel275c3c6675/).

Now that a `BinaryDecisionTree` class is available, a specific instance will be trained and evaluated on the dataset.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)
        
dt = BinaryDecisionTree(df_train, 5, "target")
dt.train()

dt_metrics = Metrics(dt.predict_df(df_test, labels=[0, 1]))
dt_metrics.print()

Without significant pre-processing and without any dimensionality reduction, the `BinaryDecisionTree` model achieves an accuracy of 87%, and a Weighted F1 of 87%. However, this is for a specific train/test split, and may be an aberration from actual expected model performance. The model should be evaluated with K-fold Cross-Validation to obtain more accurate metric values. 

In [ ]:
from sklearn.model_selection import KFold

def k_fold_eval(df, height, splits, random_state=100, verbose=False):
    """A convenience function which obtains the K-fold Cross-Validation metrics for a BinaryDecisionTree model.
    
    Keyword Arguments:
    df - The total DataFrame, which will be split into train/test subsets.
    height - The maximum height of the BinaryDecisionTree
    splits - The number of folds to use for the K-fold Cross-Validation
    random_state - The specific K-fold random state to use
    verbose - Whether or not to log additional details to console
    """
    kf_cm = [[0, 0], [0, 0]]
    k_fold = KFold(n_splits=splits, random_state=random_state, shuffle=True)
    for train_set, test_set in k_fold.split(df):
        df_train, df_test = df.iloc[train_set], df.iloc[test_set]
        dt = BinaryDecisionTree(df_train, height, "target")
        dt.train()
        if verbose:
            print("Root node condition: {0}".format(dt.root.condition))
            print("Root true_child node condition: {0}".format(dt.root.true_child.condition))
            print("Root false_child node condition: {0}".format(dt.root.false_child.condition))
        dt_cm = dt.predict_df(df_test, labels=[0, 1])
        if verbose:
            Metrics(dt_cm).print()
        kf_cm = np.add(kf_cm, dt_cm)
    print("Combined Metrics:")
    Metrics(kf_cm).print()


Now, the `BinaryDecisionTree` will be trained and evaluated again using K-fold Cross-Validation.

In [ ]:
k_fold_eval(df, 5, 4, verbose=True)

The metrics of the model after performing K-fold Cross-Validation are worse than the initial train/test split originally belied. The combined accuracy of the model, as well as the Weighted F1, is 79%. Additionally, there isn't a significant difference between recalls and precisions between the two classes, although the model has slightly better recall and accuracy for the positive class (patient doesn't have heart disease) than the negative class.

With the evaluation of the model proceeding with `verbose == True`, some basic insight into the criteria that the `BinaryDecisionTree` is using to partition the data is also provided. For convenience, explanations of the variables that are observed here are provided below.

```
thal (thalium): 1 = fixed defect; 2 = normal; 3 = reversable defect
cp (chest pain type): 0 = asymptomatic; 1 = atypical angina;  2 = non-anginal pain; 3 = typical angina
slope (the slope of the peak exercise ST segment
ca (number of major vessels (0-3) colored by flourosopy)
```

So `thalium` revealing a normal or reversable defect, `chest pain` being asmymptomatic, `age` being greater than 56, and `number of major vessels` all seem to be highly predictive of whether or not a patient has heart disease.

But no further changes will be made in an attempt to improve the `BinaryDecisionTree` model (a previous notebook of mine covers attempts at doing such [here](https://www.kaggle.com/bullardla/kernel275c3c6675/)). Instead, the `BinaryDecisionTree` class will be used as the foundation of a `BinaryRandomForest` class, to be developed below.

In [ ]:
from scipy import stats

class BinaryRandomForest:
    def __init__(self, df, num_trees, tree_dimensionality, tree_height, target_col):
        """Makes use of the BinaryDecisionTree class to allow for the construction of a Random Forest model.
        
        Keyword Arguments:
        df - The training set DataFrame
        num_trees - The number of decision trees in the Random Forest
        tree_dimensionality - The subset of variables that an individual tree should be trained on. For example, tree_dimensionality == 2 would mean each tree would be trained on a random
        subset of the input variables of size two, chosen at random
        tree_height - The maximum height of any individual tree
        target_col - The target column to model
        """
        self.df = df
        self.n = num_trees
        self.k = tree_dimensionality
        self.h = tree_height
        self.target = target_col
        self.forest = []
    
    def train(self):
        """Trains the model by initializing and training a random forest of size self.n
        """
        for i in range(self.n):
            subset = self.df.sample(frac=1, replace=True, random_state=i)
            df_subset = pd.concat([subset.drop(self.target, axis=1).sample(n=self.k, axis=1, random_state=i),subset[[self.target]]],axis=1)
            dt = BinaryDecisionTree(df_subset, self.h, self.target)
            dt.train()
            self.forest.append(dt)
            
    def predict(self, data):
        """Predicts the outcome for a specific data point.
        """
        predictions = []
        for dt in self.forest:
            predictions.append(dt.predict(data)[1])
        return ("The outcome for ({0}) is: {1}".format(str(data), stats.mode(predictions)[0][0]), stats.mode(predictions)[0][0])
    
    def predict_df(self, test_df, labels):
        """Makes a prediction for an entire testing set DataFrame and returns a confusion matrix of the predictions.
        
        Keyword Arguments:
        test_df - The testing set DataFrame
        labels - The ordering of labels for the confusion matrix to be returned
        """
        df = test_df.copy()
        df["prediction"] = df.apply(lambda row: self.predict(row.to_dict())[1], axis=1)
        return confusion_matrix(df[self.target], df["prediction"], labels=labels)
        
        
def k_fold_eval_rf(df, n_trees, tree_dim, height, splits, random_state=100, verbose=False):
    """A convenience function which obtains the K-fold Cross-Validation metrics for a BinaryRandomForest model.
    
    Keyword Arguments:
    df - The total DataFrame, which will be split into train/test subsets.
    n_trees - The number of trees in the BinaryRandomForest
    tree_dim - The number of variables a single tree is trained on.
    height - The maximum height of a BinaryDecisionTree in the BinaryRandomForest
    splits - The number of folds to use for the K-fold Cross-Validation
    random_state - The specific K-fold random state to use
    verbose - Whether or not to log additional details to console
    """
    kf_cm = [[0, 0], [0, 0]]
    k_fold = KFold(n_splits=splits, random_state=random_state, shuffle=True)
    for train_set, test_set in k_fold.split(df):
        df_train, df_test = df.iloc[train_set], df.iloc[test_set]
        rf = BinaryRandomForest(df_train, n_trees, tree_dim, height, "target")
        rf.train()
        if verbose:
            for dt in rf.forest:
                print("Root node condition: {0}".format(dt.root.condition))
        rf_cm = rf.predict_df(df_test, labels=[0, 1])
        #Metrics(rf_cm).print()
        kf_cm = np.add(kf_cm, rf_cm)
    print("Combined Metrics:")
    Metrics(kf_cm).print()


Similar to with the `BinaryDecisionTree`, there isn't too much to say about the above code that isn't already covered by the comments. However, the general intution behind a random forest model will be detailed below.

In short, a random forest model excels because the model utilizes the wisdom of the crowd. Decision trees are frequently cited as having low variance and high bias, and ultimately struggle at a lot of classification tasks due to this shortcoming. By making a prediction based on the classifications of a set of decision trees, however, a random forest overcomes many of the issues that a single decision tree faces. 

So, a random forest can be thought of as a collection of decision trees. However, if each decision tree in the random forest was trained on the same set of data, each decision tree would always make the same prediction, as all of the decision trees would be identical. To prevent this, random forests use a technique known as bagging, which dictates that each decision tree is trained on a subset of the actual training data, but with each subset being the same size as the original training set. This is accomplished by taking a subset of the data, where the subset contains only `X%` of the original dataset, and then random elements from the subset are replicated until the subset is the same size as the original training set. This helps to ensure each decision tree is biased in a certain direction, with the hope being that __each decision tree errs in a unique way, such that the colleciton of the errors cancel each other out__. 

To further ensure the above qualification, each decision tree is also presented only a subset of the original variables present in the training set. Thus, if a random forest model was to predict the age of a person based on their height, weight, and hair color, one decision tree in the model might be trained using only height and weight, while another decision tree might be trained with height and hair color, and so on.

The hyperparameters accepts by the `BinaryRandomForest` model are used to represent this behavior. The `BinaryRandomForest` has a configurable number of decision trees, `num_trees`, as well as a configurable number of variables each tree is provided with, `tree_dimensionality`. 

Using these variables, a `BinaryRandomForest` model will be trained that's made up of 10 decision trees, with each tree being trained on five variables and each tree having a height of three.

In [ ]:
#k_fold_eval_rf(df, 10, 5, 3, 4, verbose=True)

After training and evaluating a Random Forest with 10 5-dim 3-height trees, the following combined metrics are obtained.
```
Combined Metrics:
Confusion Matrix:
[[113  25]
 [ 38 127]]
Accuracy: 0.7920792079207921
Recalls: [0.8188405797101449, 0.7696969696969697]
Precisions: [0.7483443708609272, 0.8355263157894737]
Weighted F1: 0.7916661488117309
```
With `verbose == True`, the varying biases of the trees can be observed, with several of the decision trees having unique conditional values for the root nodes. Nevertheless, the model obtains an accuracy of 79% and a Weighted F1 of 79%, matching the performance of the single decision tree trained above. Will adding more trees to the Random Forest, but keeping the other hyperparameters the same, improve performance?

In [ ]:
#k_fold_eval_rf(df, 25, 5, 3, 4)
#k_fold_eval_rf(df, 50, 5, 3, 4)

The two models trained above achieved the following performance:
```
25 5-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[100  38]
 [ 21 144]]
Accuracy: 0.8052805280528053
Recalls: [0.7246376811594203, 0.8727272727272727]
Precisions: [0.8264462809917356, 0.7912087912087912]
Weighted F1: 0.8069011500974295

50 5-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[ 99  39]
 [ 19 146]]
Accuracy: 0.8085808580858086
Recalls: [0.717391304347826, 0.8848484848484849]
Precisions: [0.8389830508474576, 0.7891891891891892]
Weighted F1: 0.8105890499764261
```
Both models achieve 81% for both accuracy and their Weighted F1 scores compared to the previous models' performance of 79% for both metrics. Will performance continue to increase as more trees are added to the forest, and by how much?

In [ ]:
#k_fold_eval_rf(df, 75, 5, 3, 4)
#k_fold_eval_rf(df, 100, 5, 3, 4)

After executing the above code and waiting _several hours_ for the models to train on the dataset, the following results are obtained:
```
75 5-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[103  35]
 [ 18 147]]
Accuracy: 0.8250825082508251
Recalls: [0.7463768115942029, 0.8909090909090909]
Precisions: [0.8512396694214877, 0.8076923076923077]
Weighted F1: 0.826538321273962

100 5-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[104  34]
 [ 20 145]]
Accuracy: 0.8217821782178217
Recalls: [0.7536231884057971, 0.8787878787878788]
Precisions: [0.8387096774193549, 0.8100558659217877]
Weighted F1: 0.8229171976313556
```
With additional trees in the Random Forest, the model continues to improve, with 83% accuracy/Weighted F1 for the model with 75 trees and 82% accuracy/Weighted F1 for the model with 100 trees. The accuracy of the model improved 4% from the 79% accuracy for the Random Forest with 10 trees to the 83% accuracy for the Random Forest with 75 trees. 

It's unclear whether the marginal increase in performance warrants the orders of magnitude longer training time due to the addition of 65 trees to the Random Forest. An answer to this question would likely differ from team to team, and require additional information outside the scope of this notebook, such as time, budget, and computational resources. And, while there may always exist some team which would chose the model with the shortest training time, a general goal of making a successful model is to construct a model such that both philosophies - prioritizing shorter training time versus prioritizing model accuracy/performance - are properly satisified.

While it's evident that solely increasing the number of trees in the Random Forest isn't sufficient for significantly outperforming the single `BinaryDecisionTree` model, there are still other hyperparameters that can be tuned - `tree_dim` and `tree_height`. But it's not immediately clear how those other hyperparameters should be tweaked to improve performance. 

A few random permutations of one of the previously built Random Forests will be trained and evaluated to see how performance is affected.

In [ ]:
# Increase the max height of a tree from 3 to 5.
#k_fold_eval_rf(df, 25, 5, 5, 4)

# Decrease the max height of a tree from 3 to 2.
#k_fold_eval_rf(df, 25, 5, 2, 4)

# Increase the dimensions of trees from 5 to 8.
#k_fold_eval_rf(df, 25, 8, 3, 4)

# Decrease the dimensions of trees from 5 to 3.
#k_fold_eval_rf(df, 25, 3, 3, 4)

After testing several variations of a Random Forest model, the following metrics are obtained for the models:
```
25 5-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[100  38]
 [ 21 144]]
Accuracy: 0.8052805280528053
Recalls: [0.7246376811594203, 0.8727272727272727]
Precisions: [0.8264462809917356, 0.7912087912087912]
Weighted F1: 0.8069011500974295

25 5-dim 5-height trees:
Combined Metrics:
Confusion Matrix:
[[ 98  40]
 [ 21 144]]
Accuracy: 0.7986798679867987
Recalls: [0.7101449275362319, 0.8727272727272727]
Precisions: [0.8235294117647058, 0.782608695652174]
Weighted F1: 0.8006416018498598

25 5-dim 2-height trees:
Combined Metrics:
Confusion Matrix:
[[ 97  41]
 [ 16 149]]
Accuracy: 0.8118811881188119
Recalls: [0.7028985507246377, 0.9030303030303031]
Precisions: [0.8584070796460177, 0.7842105263157895]
Weighted F1: 0.8146257530191604

25 8-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[ 99  39]
 [ 17 148]]
Accuracy: 0.8151815181518152
Recalls: [0.717391304347826, 0.896969696969697]
Precisions: [0.853448275862069, 0.7914438502673797]
Weighted F1: 0.8174098905953587

25 3-dim 3-height trees:
Combined Metrics:
Confusion Matrix:
[[100  38]
 [ 15 150]]
Accuracy: 0.8250825082508251
Recalls: [0.7246376811594203, 0.9090909090909091]
Precisions: [0.8695652173913043, 0.7978723404255319]
Weighted F1: 0.8273348581317055
```
Ordered by their Weighted F1 scores from least to greatest, the models are:

`5-dim 5-height (80%, 80%) < 5-dim 3-height (81%, 81%) < 5-dim 2-height (81%, 82%) < 8-dim 3-height (82%, 82%) < 3-dim 3-height (83%, 83%)`

The performance of a Random Forest appears to improve with smaller trees with lower dimensionality. But why? After all, it's ill-advised to try every permutation of hyperparameters, even after making the observation that performance seems to be somewhat correlated with smaller dimesionality and tree heights. What is the intuition behind Random Forests? What makes a Random Forest (typically) achieve much better performance than an individual decision tree? Answers to these questions will provide valuable insight into choosing promising values for the hyperparameters.

A Random Forest typically performs better than a single decision tree due the ensemble of trees offering protection against any individual errors within decision trees. That is, while an individual decision tree may err, either due to overfitting, underfitting, or due to a different shortcoming, a collection of decision trees can be robust to error __if they all err in different ways__. This can be achieved if the decision trees that make up a Random Forest are uncorrelated. A Random Forest where all trees have a similar bias is no better than a single decision tree (consider a Random Forest with 100 decision trees that are all the same - the predictions made by the Random Forest will be identical with those of the single decision tree the Random Forest is comprised of). 

Of course, if all the decision trees in a Random Forest are trained on the same dataset, the trees will be identical in structure. Each tree in the forest thus needs to be trained on a subset of the original dataset, which is accomplished through bootstrap aggregating (bagging) and through training each tree on only a subset of the input variables. 

The purpose of bootstrap aggregating, instead of simply subsampling the training set, is to discourage training a model which accurately maps the training set but fails to generalize to the testing set. For this particular dataset, there are only ~300 entries in the combined dataset, while there are ~300,000,000 people in the United State population. This dataset represents only 0.0001% of the United States population, and could thus feature trends that aren't indicative of the larger population. In the entire U.S. population, it may be observed that men and women are at equal risk of heart disease, but a sample size of only 300 may belie this and have all male samples as having heart disease and all female samples as not having heart disease.  For such a subset a decision tree could easily obtain 100% accuracy, but would be expected to only obtain ~50% accuracy when applied to the entire U.S. population. 

In the above scenario, it can be readily understood how sampling a subset of input variables might improve testing accuracy. Any decision tree trained on a group of variables including the `gender` variable will have only a single root node that splits the data by gender and achieves maximum information gain as both subsets will have zero entropy (male subset will all be `heart_disease` and female subset will all be `no_heart_disease`). But a decision tree that wasn't trained on this variable would not err in the same way. Individually, it might achieve worse performance on the training set - and may even cause the entire Random Forest to achieve worse performance on the training set - but it would (hopefully) achieve better performance on the testing set and therefore boost the performance of the Random Forest. This is the intuition behind why a collection of uncorrelated models will often achieve better performance than any of the individual models the collection is comprised of.

But it still might not be obvious why bagging is used for the training set entries instead of training each decision tree on a different subset of the entries. What is the benefit of selection with replacement versus selection without replacement?

While sampling without replacement can lead to an ensemble model with high variance, the theory behind bagging - sampling with replacement - is that such a technique reduces the variance of an ensemble model. Intuitively, by sampling with replacement, a dataset which is a sample distribution from a larger population can be used to create additional sample disributions that also may have plausibly occurred from the sampled population. The input variables in each bagged sample will have slight variations in their distributions, causing each decision tree to be fitted on a different potential sample distribution from the original population.

Perhaps more intuitively, bagging can be viewed as a sampling methood which adds varying weights to different data points. For a training set with the entries `[1, 2, 3, 4, 5]`, a bagged permutation of `[1, 1, 1, 3, 3]` can be thought of as `{1: 3, 2: 0, 3: 2, 4: 0, 5: 0}`. By weighting entries differently across decision trees in the Random Forest, the resulting trees will in theory be uncorrelated, each tree erring in a different way such that the sum of the errors cancel out for any given input.

Given this intuition behind the usefulness of a Random Forest model, a logical starting point for a Random Forest model would be one made up of many decision trees of varying heights and dimensions. But for a Random Forest with only one type of decision tree, what Random Forest model will perform the best? Will the observation of a Random Forest made up of shorter, lower dimensionality decision trees providing the best overall perfomance hold?

In [ ]:
import matplotlib.pyplot as plt

# This takes several days to complete. The insight gathered from it is included below.
#
#for height in range(1, 4):
#    for dim in range(1, 14):
#        print((height, dim))
#        k_fold_eval_rf(df, 25, dim, height, 4)

f1_scores_25_1 = [.7218, .7652, .8223, .8100, .8159, .8026, .7958, .8026, .7928, .7930, .7833, .7825, .7729]
f1_scores_25_2 = [.7115, .7834, .8188, .8276, .8146, .7959, .8002, .8188, .8242, .8150, .8120, .8184, .8211]
f1_scores_25_3 = [.7276, .7769, .8273, .8130, .8069, .8273, .8307, .8174, .8200, .8273, .8339, .8140, .8177]
f1_scores_10_1 = [.6554, .7958, .7851, .7851, .7950, .7785, .7819, .8151, .8150, .7984, .7984, .8083, .7719]
f1_scores_10_2 = [.6548, .7725, .8023, .7985, .7986, .7852, .7964, .8123, .8088, .8096, .8100, .8119, .7851]
f1_scores_10_3 = [.6600, .7593, .7892, .7924, .7917, .8152, .8185, .8251, .8485, .8352, .8387, .8325, .8261]
f1_scores_50_1 = [.7575, .7955, .8320, .8092, .8259, .8156, .8193, .8329, .8195, .8092, .7889, .7727, .7723]
f1_scores_50_2 = [.7399, .7726, .8215, .8215, .8245, .8153, .7959, .8218, .8127, .8032, .8184, .8150, .8041]
f1_scores_50_3 = [.7498, .7795, .8234, .8064, .8106, .8237, .8428, .8399, .8297, .8365, .8464, .8234, .8271]

fig, ax = plt.subplots(2, figsize=(12, 12))

ax[0].plot([i + 1 for i in range(len(f1_scores_10_3))], f1_scores_10_3, color="red", linestyle='dotted', label="Trees == 10, Height == 3")
ax[0].plot([i + 1 for i in range(len(f1_scores_10_2))], f1_scores_10_2, color="blue", linestyle='dotted', label="Trees == 10, Height == 2")
ax[0].plot([i + 1 for i in range(len(f1_scores_10_1))], f1_scores_10_1, color="black", linestyle='dotted', label="Trees == 10, Height == 1")
ax[0].plot([i + 1 for i in range(len(f1_scores_25_3))], f1_scores_25_3, color="red", linestyle='dashed', label="Trees == 25, Height == 3")
ax[0].plot([i + 1 for i in range(len(f1_scores_25_2))], f1_scores_25_2, color="blue", linestyle='dashed', label="Trees == 25, Height == 2")
ax[0].plot([i + 1 for i in range(len(f1_scores_25_1))], f1_scores_25_1, color="black", linestyle='dashed', label="Trees == 25, Height == 1")
ax[0].plot([i + 1 for i in range(len(f1_scores_50_3))], f1_scores_50_3, color="red", label="Trees == 50, Height == 3")
ax[0].plot([i + 1 for i in range(len(f1_scores_50_2))], f1_scores_50_2, color="blue", label="Trees == 50, Height == 2")
ax[0].plot([i + 1 for i in range(len(f1_scores_50_1))], f1_scores_50_1, color="black", label="Trees == 50, Height == 1")
ax[0].set_title("Random Forest Performance")
ax[0].grid()
ax[0].legend()
ax[0].set_ylabel("Weighted F1 Scores")

ax[1].plot([i + 1 for i in range(13)], [(f1_scores_10_1[i] + f1_scores_25_1[i] + f1_scores_50_1[i])/3 for i in range(13)],
         color="red", label="Average of Height == 1")
ax[1].plot([i + 1 for i in range(13)], [(f1_scores_10_2[i] + f1_scores_25_2[i] + f1_scores_50_2[i])/3 for i in range(13)],
         color="blue", label="Average of Height == 2")
ax[1].plot([i + 1 for i in range(13)], [(f1_scores_10_3[i] + f1_scores_25_3[i] + f1_scores_50_3[i])/3 for i in range(13)],
         color="green", label="Average of Height == 3")
ax[1].plot([i + 1 for i in range(13)], [(f1_scores_10_1[i] + f1_scores_10_2[i] + f1_scores_10_3[i])/3 for i in range(13)],
         color="red", linestyle='dashed', label="Average of Trees == 10")
ax[1].plot([i + 1 for i in range(13)], [(f1_scores_25_1[i] + f1_scores_25_2[i] + f1_scores_25_3[i])/3 for i in range(13)],
         color="blue", linestyle='dashed', label="Average of Trees == 25")
ax[1].plot([i + 1 for i in range(13)], [(f1_scores_50_1[i] + f1_scores_50_2[i] + f1_scores_50_3[i])/3 for i in range(13)],
         color="green", linestyle='dashed', label="Average of Trees == 50")

ax[1].set_xlabel("Number of Dimensions")
ax[1].set_ylabel("Weighted F1 Scores")
ax[1].legend()
ax[1].grid()
plt.show()

There are several observations worth mentioning based on the obtained data.
1. At higher dimensionality, the height of trees in a Random Forest is positively correlated with the Weighted F1 score of the model.
2. At lower dimensionality, the number of trees in a Random Forest is positively correlated with the Weighted F1 score of the model.
3. The model with the best overall performance across dimensionalities is the Random Forest with 50 trees, each with a height of three.
4. The model that achieves the highest Weighted F1 score at any single level of dimensionality is the Random Forest with 10 trees, each with a height of tree (though this may be an outlier).

None of the observations are particularly surprising, although it's important to highlight that Random Forest models are a lot more robust to overfitting even when their decision trees have heights that lead to significant individual overfitting. Again, the individual errors of tall, overfitting trees will (theoretically) be cancelled out in aggregate, so it's unlikely that performance of the Random Forest will decrease once a certain height is reached despite this being true of a single decison tree model.

The previous observation that a Random Forest of shorter trees trained on fewer dimensions achieves better performance than a Random Forest with taller, higher dimensionality trees has been disproved with this more thorough testing. Nevertheless, the apparent marginal performance gain of most of the models between dimensionalities of 3 and 13 may not warrant the additional training time required to compile the model, though Random Forests with taller trees seem to benefit more from additional dimensions than those with shorter trees.

A few of the trained models were able to achieve a Weighted F1 score of 85%, 6% higher than the 79% that a single decision tree model was able to achieve. But is this the best performance a Random Forest model can achieve on the dataset? Given the previous observation that Random Forest performance seems to continuously improve as the trees the model is composed of get taller, a Random Forest model will be trained at a range of different heights to see hoow performance fluctuates.

In [ ]:
# This takes days to run. The output from running it is shown below.
#for height in range(1, 11):
#    k_fold_eval_rf(df, 50, 10, height, 4)

f1s = [.8092, .8032, .8365, .8239, .8365, .8203, .8234, .8234, .8265, .8231]

plt.plot([i+1 for i in range(10)], f1s)
plt.xlabel("Random Forest Tree Heights")
plt.ylabel("Weighted F1 Scores")
plt.title("Performance of Random Forests with 100 10-dim trees")
plt.grid()
plt.show()

Interestingly, the performance of the Random Forest model compsoed of 50 10-dim trees appears to peak when the trees have a height between 3 and 5, inclusive. This may be a result of there not being enough trees in the Random Forest to compensate for the decision trees overfitting as they get taller. It's also possible the intuition that a Random Forest is robust to overfitting is erroneous, although this is unlikely to be the case. Perhaps the results are simply noisy.

To rule out some of these possibilities, a different model will be trained on a range of tree heights. Ideally, many Random Forest models would be trained on a range of tree heights to get a more reliable trend, but due to the time constraints of training so many Random Forest models only two additional models will be evaluated to see how performance correlates with tree heights.

In [ ]:
'''
for dims in [5, 7, 10]:
    for height in range(1, 11):
        k_fold_eval_rf(df, 100, 5, height, 4)
'''

f1s_10dim = [.8092, .8032, .8365, .8239, .8365, .8203, .8234, .8234, .8265, .8231]
f1s_7dim = [.8261, .8188, .8399, .8295, .8299, .8297, .8329, .8327, .8227, .8227]
f1s_5dim = [.8268, .8119, .8229, .8234, .8200, .8198, .8134, .8166, .8164, .8164]
plt.plot([i+1 for i in range(10)], f1s_10dim, label="10-dim Trees", color="red")
plt.plot([i+1 for i in range(10)], f1s_7dim, label="7-dim Trees", color="blue")
plt.plot([i+1 for i in range(10)], f1s_5dim, label="5-dim Trees", color="green")
plt.xlabel("Random Forest Tree Heights")
plt.ylabel("Weighted F1 Scores")
plt.title("Performance of Random Forests with 100 trees")
plt.grid()
plt.legend()
plt.show()

Interestingly, the average performance of all of models appears to peak when each tree has a height of three, and each model achieves its worst performance when each tree has a height of two. Why the performance happens to drop for each model when the height of trees are increased from one to two before increasing again when the height of the trees are increased from two to three is unclear, especially since the previous tests suggested that the performance of Random Forest models was positively correlated with the height of the constituent trees.

Also peculiar is the strengthened observation that performance is not correlated with tree height for these three models, with performance for all of the Random Forest models peaking when their trees have a height between three and five. Performance of these models isn't linearly correlated with the number of dimensions, either, with the 7-dim tree Random Forest model outperforming the other two models that were evaluated. 

After these findings, and due to the lack of resources necessary to train more 100 tree Random Forest models for this part (the three above models took over a week to evaluate) it is not obvious what the next step should be to improve performance of the Random Forest model further. The maximum obtained Weighted F1 score is still 85%, and further testing makes it unclear whether increasing the number of trees, the heights of the trees, or the number of dimensions each tree is trained on in the Random Forest will improve performance.

As the intuition behind the Random Forest model can be summed up succinctly with that they "use the wisdom of crowds", and since a performant Random Forest requires that each individual tree is uncorrelated, perhaps varying the height and dimensionality of each tree in the Random Forest will lead to better results. That is, instead of compiling a Random Forest made up of trees of height `X` and dimensionality `Y`, a Random Forest model can be trained where each tree has a height between `X1 ... X2`, and a dimensionality between `Y1 ... Y2`. This might lead to the trees in the Random Forest being less correlated with each other than they were in the previously evaluated models.

This theory will be tested below by building a Random Forest model with 100 trees, with each tree having a dimesionality between five and ten and a height between three and five.

In [ ]:
class MixedBinaryRandomForest(BinaryRandomForest):
    def __init__(self, df, num_trees, tree_dimensionality_range, tree_height_range, target_col, seed=100):
        """Makes use of the BinaryDecisionTree class to allow for the construction of a Random Forest model.
        
        Keyword Arguments:
        df - The training set DataFrame
        num_trees - The number of decision trees in the Random Forest
        tree_dimensionality_range - A range of subsets of variables that an individual tree should be trained on. For example, tree_dimensionality_range == (2, 4) would mean each tree would be trained on a random
        subset of the input variables of size between two and four, chosen at random
        tree_height_range - A range of maximum heights of any individual tree. For example, tree_height_range == (2, 4) would mean each tree would have a height between two and four, randomly chosen.
        target_col - The target column to model
        seed - The random seed to use for deterministic purposes of the notebook
        """
        self.df = df
        self.n = num_trees
        self.k_range = tree_dimensionality_range
        self.h_range = tree_height_range
        self.target = target_col
        self.forest = []
        np.random.seed(seed)
    
    def train(self):
        """Trains the model by initializing and training a random forest of size self.n
        """
        for i in range(self.n):
            subset = self.df.sample(frac=1, replace=True, random_state=i)
            df_subset = pd.concat([subset.drop(self.target, axis=1).sample(n=np.random.randint(self.k_range[0], self.k_range[1] + 1), axis=1, random_state=i),subset[[self.target]]],axis=1)
            dt = BinaryDecisionTree(df_subset, np.random.randint(self.h_range[0], self.h_range[1] + 1), self.target)
            dt.train()
            self.forest.append(dt)
            
def k_fold_eval_mrf(df, n_trees, tree_dim_range, height_range, splits, random_state=100, verbose=False):
    """A convenience function which obtains the K-fold Cross-Validation metrics for a MixedBinaryRandomForest model.
    
    Keyword Arguments:
    df - The total DataFrame, which will be split into train/test subsets
    n_trees - The number of trees in the BinaryRandomForest
    tree_dim_range - The range of the number of variables a single tree is trained on, chosen at random
    height - The range of maximum heights of a BinaryDecisionTree in the MixedBinaryRandomForest
    splits - The number of folds to use for the K-fold Cross-Validation
    random_state - The specific K-fold random state to use
    verbose - Whether or not to log additional details to console
    """
    kf_cm = [[0, 0], [0, 0]]
    k_fold = KFold(n_splits=splits, random_state=random_state, shuffle=True)
    for train_set, test_set in k_fold.split(df):
        df_train, df_test = df.iloc[train_set], df.iloc[test_set]
        rf = MixedBinaryRandomForest(df_train, n_trees, tree_dim_range, height_range, "target")
        rf.train()
        if verbose:
            for dt in rf.forest:
                print("Root node condition: {0}".format(dt.root.condition))
        rf_cm = rf.predict_df(df_test, labels=[0, 1])
        #Metrics(rf_cm).print()
        kf_cm = np.add(kf_cm, rf_cm)
    print("Combined Metrics:")
    Metrics(kf_cm).print()

In [ ]:
# Build the model. The results of building the model are provided below.
#
#k_fold_eval_mrf(df, 100, (5, 10), (3, 5), 4)

```
Combined Metrics:
Confusion Matrix:
[[103  35]
 [ 14 151]]
Accuracy: 0.8382838283828383
Recalls: [0.7463768115942029, 0.9151515151515152]
Precisions: [0.8803418803418803, 0.8118279569892473]
Weighted F1: 0.8401050663135695
```

The results of the mixed Random Forest model are decent relative to the other models trained thus far, but the model is still only able to achieve a Weighted F1 score of 84%. 

`BinaryRandomForest` and `MixedBinaryRandomForest` don't seem to be promising candidates for a model able to achieve higher than a 90% Weighted F1 score on this dataset, so different Random Forest implentations will need to be considered. The intention behind enhancing the original `BinaryRandomForest` by creating a `MixedBinaryRandomForest` was to add diversity to the model with the hope of therefore reducing correlation amongst the constituent trees. However, while each tree in the `MixedBinaryRandomForest` can have a range of heights and a range of dimensionalities, each tree in the model will always split based on the same dimensions. That is, a tree in the model trained on the three dimensions `age`, `sex`, and `chol` will always make splits based on those same three dimensions. It will never split the dataset based on any of the other ten possible input dimensions - other trees will be needed to make decisions based off of those other dimensions.

Instead of training a single binary decision tree based off of the same `k` dimensions for every node, what if a decision tree made a split based off a random `k` dimensions for every node? That is, the root node of a three-dimensional tree may split the dataset based on the variables `age`, `sex`, and `chol`, but a child node may make a split based off of the variables `fbs`, `slope`, and `thal`. In theory, this should further reduce the correlation among each decision tree in the Random Forest and therefore lead to better overall performance of the model. 

Classes for this new Random Forest model will be defined below.

In [ ]:
class BDTShuffledNode(BDTNode):
    def __init__(self, df, cols, target_col, level):
        """An individual node in a binary decision tree. It's either a leaf node or has two children, where true_child is the child of the node matching the BDTShuffledNode's condition, 
        and false_child is the child not matching the given condition.
         
        Keyword Arguments:
        df - The filtered DataFrame of this specific node, where the DataFrame will be a subset of the parent node's DataFrame after conditional filtering.
        cols - The input columns which the specific node uses to make a split decision.
        target_col - The target variable to be used for training. As this tree is only intended for use with binary classification tasks, there can only be one target variable.
        level - The level of this node in the BinaryDecisionTree, with 0 denoting the root node.
        """
        self.df = df
        self.input_cols = cols
        self.target = target_col
        self.level = level
        self.condition = None
        self.true_child = None
        self.false_child = None
        self.ig = 0
        

class ShuffledBinaryDecisionTree(BinaryDecisionTree):
    def __init__(self, df, dim, height, target_col):
        """A binary decision tree. The qualification of binary denotes that the decision tree is for binary classification tasks, and not categorical classification tasks, due to 
        how entropy is calculated in the BDTShuffledNode.
        
        Keyword Argumments:
        df - The DataFrame to use for training the ShuffledBinaryDecisionTree. Note that the ShuffledBinaryDecisionTree will not split the data up into testing and training sets - that must be done beforehand.
        dim - The dimensionality of the ShuffledBinaryDecisonTree. Determines how many values each individual node will use as criteria for splitting.
        height - The maximum height of the ShuffledBinaryDecisionTree. This helps prevent overfitting in lieu of pruning.
        target_col - The target variable to be used for training. As this tree is only intended for use with binary classification tasks, there can only be one target variable.
        """
        self.df = df
        self.height = height
        self.dim = dim
        self.target = target_col
        self.input_cols = list(df.columns)
        self.input_cols.remove(target_col)
        self.root = BDTShuffledNode(df, np.random.choice(self.input_cols, dim), self.target, 0)
             
    def _branch(self, node):
        """A utility function which comprises the core logic of the BinaryDecisionTree. Branches a single BDTNode into two child nodes by greedily partitioning the DataFrame into
        two subsets based on the condtional split which results in the most information gain.
        
        Returns:
        True - if self._branch(node) paritioned node's data into two subsets
        False if self._branch(node) resulted in node becoming a leaf node, either due to the maximum height of the tree being reached or a failure to find a partition leading to
        information gain.
        """
        if node.level == self.height:
            return False
        df = node.df
        input_cols = node.input_cols
        for col in input_cols:
            for unique_value in df[col].unique():
                subsets = [("{0} == {1}".format(col, unique_value), df.loc[df[col] == unique_value], df.loc[df[col] != unique_value]), 
                           ("{0} >= {1}".format(col, unique_value), df.loc[df[col] >= unique_value], df.loc[df[col] < unique_value])] if node.df[col].dtype in ["int64", 
                           "float64"] else [("{0} == {1}".format(col, unique_value), df.loc[df[col] == unique_value], df.loc[df[col] != unique_value])]
                for condition, c1, c2 in subsets:
                    old_true_child, old_false_child, old_condition = node.true_child, node.false_child, node.condition
                    node.condition = condition
                    node.true_child = BDTShuffledNode(c1, np.random.choice(self.input_cols, self.dim), node.target, node.level + 1)
                    node.false_child = BDTShuffledNode(c2,  np.random.choice(self.input_cols, self.dim), node.target, node.level + 1)
                    ig = node.get_information_gain()
                    if ig > node.ig:
                        node.ig = ig
                    else:
                        node.condition = old_condition
                        node.true_child, node.false_child = old_true_child, old_false_child
        return node.ig != 0
    
    
class ShuffledBinaryRandomForest(MixedBinaryRandomForest):
    def __init__(self, df, num_trees, tree_dimensionality_range, tree_height_range, target_col, r_seed=100):
        super().__init__(df, num_trees, tree_dimensionality_range, tree_height_range, target_col, seed=r_seed)
    
    def train(self):
        """Trains the model by initializing and training a random forest of size self.n
        Same behavior as the parent class MixedBinaryRandomForest, except that trees in the forest are ShuffledBinaryDecisionTrees, and not plain BinaryDecisionTrees
        """
        for i in range(self.n):
            subset = self.df.sample(frac=1, replace=True, random_state=i)
            dt = ShuffledBinaryDecisionTree(df, np.random.randint(self.k_range[0], self.k_range[1] + 1), np.random.randint(self.h_range[0], self.h_range[1] + 1), self.target)
            dt.train()
            self.forest.append(dt)

A `ShuffledBinaryRandomForest` model will be trained below, and the results will be provided.

In [ ]:
def k_fold_eval_srf(df, n_trees, tree_dim_range, height_range, splits, random_state=100, verbose=False):
    """A convenience function which obtains the K-fold Cross-Validation metrics for a ShuffledBinaryRandomForest model.
    
    Keyword Arguments:
    df - The total DataFrame, which will be split into train/test subsets
    n_trees - The number of trees in the BinaryRandomForest
    tree_dim_range - The range of the number of variables a single tree is trained on, chosen at random
    height - The range of maximum heights of a BinaryDecisionTree in the MixedBinaryRandomForest
    splits - The number of folds to use for the K-fold Cross-Validation
    random_state - The specific K-fold random state to use
    verbose - Whether or not to log additional details to console
    """
    kf_cm = [[0, 0], [0, 0]]
    k_fold = KFold(n_splits=splits, random_state=random_state, shuffle=True)
    for train_set, test_set in k_fold.split(df):
        df_train, df_test = df.iloc[train_set], df.iloc[test_set]
        rf = ShuffledBinaryRandomForest(df_train, n_trees, tree_dim_range, height_range, "target")
        rf.train()
        if verbose:
            for dt in rf.forest:
                print("Root node condition: {0}".format(dt.root.condition))
        rf_cm = rf.predict_df(df_test, labels=[0, 1])
        #Metrics(rf_cm).print()
        kf_cm = np.add(kf_cm, rf_cm)
    print("Combined Metrics:")
    Metrics(kf_cm).print()
    
#k_fold_eval_srf(df, 100, (5, 10), (3, 5), 4)

```
Combined Metrics:
Confusion Matrix:
[[115  23]
 [  8 157]]
Accuracy: 0.8976897689768977
Recalls: [0.8333333333333334, 0.9515151515151515]
Precisions: [0.9349593495934959, 0.8722222222222222]
Weighted F1: 0.8984055826872043
```

This `ShuffledBinaryRandomForest` model is able to achieve an accuracy and a Weighted F1 score of 90%, while the `MixedBinaryRandomForest` model with the same input parameters was only able to achieve 84% for both of those metrics. This is 11% higher than the 79% that the single decision tree model was able to obtain. These results are impressive, but why is this model able to perform significantly better than the `BinaryRandomForest` and the `MixedBinaryRandomForest`?

Let's first compare the performance of a normal `BinaryDecisionTree` to a `ShuffledBinaryDecisionTree` to help elucidate this.

In [ ]:
def k_fold_eval_sbt(df, height, dim, splits, random_state=100, verbose=False):
    """A convenience function which obtains the K-fold Cross-Validation metrics for a ShuffledBinaryDecisionTree model.
    
    Keyword Arguments:
    df - The total DataFrame, which will be split into train/test subsets.
    height - The maximum height of the BinaryDecisionTree
    dim - The dimensionality of the ShuffledBinaryDecisionTree
    splits - The number of folds to use for the K-fold Cross-Validation
    random_state - The specific K-fold random state to use
    verbose - Whether or not to log additional details to console
    """
    kf_cm = [[0, 0], [0, 0]]
    k_fold = KFold(n_splits=splits, random_state=random_state, shuffle=True)
    for train_set, test_set in k_fold.split(df):
        df_train, df_test = df.iloc[train_set], df.iloc[test_set]
        dt = ShuffledBinaryDecisionTree(df_train, dim, height, "target")
        dt.train()
        if verbose:
            print("Root node condition: {0}".format(dt.root.condition))
            print("Root true_child node condition: {0}".format(dt.root.true_child.condition))
            print("Root false_child node condition: {0}".format(dt.root.false_child.condition))
        dt_cm = dt.predict_df(df_test, labels=[0, 1])
        if verbose:
            Metrics(dt_cm).print()
        kf_cm = np.add(kf_cm, dt_cm)
    print("Combined Metrics:")
    Metrics(kf_cm).print()

''' 
df_subset = pd.concat([df.drop("target", axis=1).sample(3, axis=1, random_state=100), df[["target"]]],axis=1)

for height in range(1, 11):
    print("BinaryDecisionTree:")
    k_fold_eval(df_subset, height, 4)
    print("ShuffledBinaryDecisionTree:")
    k_fold_eval_sbt(df, height, 3, 4)
'''

bdt_f1s = [.6928, .7176, .6916, .6875, .7000, .7000, .7000, .7000, .7000, .7000]
sbdt_f1s = [.7453, .7468, .7326, .7559, .7773, .7508, .7556, .7588, .7302, .7526]
plt.plot([i+1 for i in range(len(bdt_f1s))], bdt_f1s, label="BinaryDecisionTree", color="red")
plt.plot([i+1 for i in range(len(sbdt_f1s))], sbdt_f1s, label="ShuffledBinaryDecisionTree", color="blue")
plt.legend()
plt.grid()
plt.xlabel("Tree Height")
plt.ylabel("Weighted F1 Score")
plt.title("Three-Dimensional Decision Tree Performance")
plt.show()

Firstly, we can see that the `ShuffledBinaryDecisionTree` significantly outperforms the `BinaryDecisionTree` at all tree heights. Secondly, the `BinaryDecisionTree` performance plateaus once a height of five is reached as the there are no more splits that would lead to information gain based on the three variables the model is restricted to. The `ShuffledBinaryDecisionTree` doesn't suffer from this same flaw - even though each node in the tree has the same dimensionality as each node in the `BinaryDecisionTree`, each node is presented with a random set of variables such that there is a chance that all thirteen input variables will be used by the tree to make splits for information gain. The `ShuffledBinaryDecisionTree` isn't restricted in its information in the same rigid manner that the `BinaryDecisionTree` is.

However, a `ShuffledBinaryDecisionTree` will not perform the same as a `BinaryDecisionTree` trained on all possible dimensions. While both trees have the potential to be trained on all input variables, the `BinaryDecisionTree` will always make the split that leads to the most information gain across all input variables. The `ShuffledBinaryDecisionTree`, meanwhile, will make the split that leads to the most information gain across only the variables the specific node was provided.

The implication of this behavior is that the `ShuffledBinaryDecisionTree` will have a lot more variance in performance than the `BinaryDecisionTree`. It has the potential to make sub-optimal splits in the training set, but this may lead to better modeling of the data in the testing set. A `BinaryDecisionTree` model trained on only a subset of the input variables, meanwhile, will have poor performance if it is provided with variables that are noisy or only loosely correlated with the target variable. 

Therefore, a `ShuffledBinaryDecisionTree` will - in general - have significantly better performance than a `BinaryDecisionTree` of the same dimensionality, but more variance in performance than a `BinaryDecisionTree` trained on all input variables. This is why the `BinaryDecisionTree` provided with all input variables was able to achieve a Weighted F1 score of 79%, while the `ShuffledBinaryDecisionTree` trained above was only able to achieve a maximum Weighted F1 score of 78%.

But what would happen if the predictions of many `ShuffledBinaryDecisionTrees` were aggregated? Then, in theory, the bias of a single tree would be cancelled out by the other trees in the aggregation as long as they tended to be biased in different ways. (Of course, then, you'd have a Random Forest).

The `ShuffledRandomForest` is the typical manner in which a Random Forest model will be implemented in most libraries, and the results obtained previously in this noteboook help to reflect why. The `ShuffledRandomForest` with 100 trees was able to obtain a Weighted F1 score of 90% on the dataset. Now, as a final test, the number of trees in the same `ShuffledRandomForest` will be increased to 200 to see how performance then changes.

In [ ]:
#k_fold_eval_srf(df, 200, (5, 10), (3, 5), 4)
#k_fold_eval_srf(df, 200, (3, 5), (3, 5), 4)
#k_fold_eval_srf(df, 200, (3, 5), (5, 7), 4)

```
k_fold_eval_srf(df, 200, (5, 10), (3, 5), 4)

Combined Metrics:
Confusion Matrix:
[[116  22]
 [  6 159]]
Accuracy: 0.9075907590759076
Recalls: [0.8405797101449275, 0.9636363636363636]
Precisions: [0.9508196721311475, 0.8784530386740331]
Weighted F1: 0.9082974904871546



k_fold_eval_srf(df, 200, (3, 5), (3, 5), 4)

Combined Metrics:
Confusion Matrix:
[[117  21]
 [  7 158]]
Accuracy: 0.9075907590759076
Recalls: [0.8478260869565217, 0.9575757575757575]
Precisions: [0.9435483870967742, 0.88268156424581]
Weighted F1: 0.908179287660703



k_fold_eval_srf(df, 200, (3, 5), (5, 7), 4)

Combined Metrics:
Confusion Matrix:
[[129   9]
 [  0 165]]
Accuracy: 0.9702970297029703
Recalls: [0.9347826086956522, 1.0]
Precisions: [1.0, 0.9482758620689655]
Weighted F1: 0.9704033543455767
```

The `ShuffledBinaryRandomForest` models composed of trees with heights of `(3, 5)` inclusive achieved an accuracy and Weighted F1 score of 91%, while the `ShuffledBinaryRandomForest` model composed of trees with heights of `(5, 7)` inclusive achieved an accuracy and Weighted F1 score of 97%. This is 18% higher than the best performing `BianryDecisionTree` model, and 11% higher than the best performing `BinaryRandomForest` or `MixedBinaryRandomForest` model. This is a remarkable improvement, and it's possible that a `ShuffledBinaryRandomForest` model with more trees with greater heights would perform even better.

These results highlight the advantages of a Random Forest model over a simple Decision Tree model, but also what specific features a Random Forest model needs to have to accurately model a dataset. A Random Forest should be made up of many uncorrelated trees, with each individual tree contributing some useful information to the overall model such that the average of all predictions made by the model is accurate for any given input.